# Indexing by Station

While I am storing data on the level of *trips*, I need to access it on the level of *stations*. Since the search strategy for determining which bikes correspond with which stations is complex, and would require a full scan of the data, I need to speed this up by precomputing an index.

I already did that work in `08` and `09` while generating counts of things, so I in part reuse that code here.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import requests
import io
import zipfile
from tqdm import tqdm

In [2]:
stations = pd.read_csv("../data/final/june_22_station_metadata.csv", index_col=0)

In [3]:
june_22 = pd.read_csv("../data/final/all_june_22_citibike_trips.csv", index_col=0)

## Outbound bike trips

The following cell generates a dict of stations by bikes which began the day there.

In [4]:
outbound_station_bikesets = {station_id: [] for station_id in stations.index}

for bike_id in tqdm(np.unique(june_22['bikeid'])):
    first_trip_station = june_22[june_22['bikeid'] == bike_id].sort_values(by='starttime').iloc[0]['start station id']
    outbound_station_bikesets[first_trip_station].append(bike_id)

100%|████████████████████████████████████| 6493/6493 [00:05<00:00, 1096.73it/s]


The index is structured like so:

    {<station id>: [<bike id>, <bike id>, ...]}

Ok great, but now we need to convert that set of bike ids to sets of trip ids.

In [5]:
import functools
outbound_bike_trip_index = dict()

for station_id, bike_id_list in tqdm(outbound_station_bikesets.items()):
    trip_set = []
    for bike_id in bike_id_list:
        trip_set.append(list(june_22[june_22['bikeid'] == bike_id].index.values))
    if len(trip_set) > 0:
        trip_set = functools.reduce(lambda x,y: x+y, trip_set)
    outbound_bike_trip_index[station_id] = trip_set

100%|███████████████████████████████████████| 474/474 [00:02<00:00, 163.16it/s]


And there you have it.

In [9]:
outbound_bike_trip_index[72]

[1005037,
 1005697,
 1034787,
 1035581,
 1005015,
 1006619,
 1007494,
 1010797,
 1012207,
 1013222,
 1013964,
 1015458,
 1023771,
 1024868,
 1028162,
 1030868,
 1032378,
 1047538,
 1049546,
 1051784,
 210,
 1016277,
 1027213,
 1032934,
 1036287]

BEGIN TEMP

In [29]:
# This is what outbounds from Penn Valet look like when we select them using the just-build index.
penn_by_index = june_22[june_22.index.isin(outbound_bike_trip_index[3230])]
penn_by_index.head(1)

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
1006021,16396,1956.0,515,40.760094,-73.994618,W 43 St & 10 Ave,1,3230,40.751284,-73.996924,Penn Station Valet,6/22/2016 06:01:37,6/22/2016 06:06:50,312,Subscriber


In [33]:
# It is not correctly picking up the first bike id we corrected, 15636.
# All eight trips are present, including the eight rebalancing trip that was not accounted for when we began this exercise.
penn_by_index[penn_by_index['bikeid'] == 15636]

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
1009289,15636,1976.0,153,40.752062,-73.981632,E 40 St & 5 Ave,1,3230,40.751284,-73.996924,Penn Station Valet,6/22/2016 07:40:55,6/22/2016 07:50:21,566,Subscriber
1009990,15636,1959.0,3236,40.758985,-73.993800,W 42 St & Dyer Ave,1,153,40.752062,-73.981632,E 40 St & 5 Ave,6/22/2016 07:51:39,6/22/2016 08:03:35,716,Subscriber
1010891,15636,1969.0,513,40.768254,-73.988639,W 56 St & 10 Ave,1,3236,40.758985,-73.993800,W 42 St & Dyer Ave,6/22/2016 08:04:34,6/22/2016 08:09:39,304,Subscriber
1011481,15636,1945.0,470,40.743453,-74.000040,W 20 St & 8 Ave,1,513,40.768254,-73.988639,W 56 St & 10 Ave,6/22/2016 08:11:43,6/22/2016 08:26:04,861,Subscriber
1012800,15636,1991.0,534,40.702551,-74.012723,Water - Whitehall Plaza,1,470,40.743453,-74.000040,W 20 St & 8 Ave,6/22/2016 08:27:26,6/22/2016 08:46:20,1133,Subscriber
1017376,15636,1984.0,304,40.704633,-74.013617,Broadway & Battery Pl,1,534,40.702551,-74.012723,Water - Whitehall Plaza,6/22/2016 09:16:53,6/22/2016 09:18:46,113,Subscriber
1038641,15636,1976.0,157,40.690893,-73.996123,Henry St & Atlantic Ave,1,2000,40.702551,-73.989402,Front St & Washington St,6/22/2016 17:18:53,6/22/2016 17:27:45,531,Subscriber
115,15636,0.0,2000,40.702551,-73.989402,Front St & Washington St,3,304,40.704633,-74.013617,Broadway & Battery Pl,6/22/2016 13:16:19,6/22/2016 13:21:19,300,Rebalancing


In [34]:
# This one, however, is still a problem.
penn_by_index[penn_by_index['bikeid'] == 17075]

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype


In [37]:
june_22[june_22['bikeid'] == 17075].sort_values(by='starttime', ascending=False)

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
1059518,17075,1967.0,3160,40.778968,-73.973747,Central Park West & W 76 St,2,520,40.759923,-73.976485,W 52 St & 5 Ave,6/22/2016 22:53:58,6/22/2016 23:07:22,803,Subscriber
278,17075,0.0,520,40.759923,-73.976485,W 52 St & 5 Ave,3,514,40.760875,-74.002777,12 Ave & W 40 St,6/22/2016 20:54:39,6/22/2016 21:08:39,840,Rebalancing
1048279,17075,1988.0,514,40.760875,-74.002777,12 Ave & W 40 St,1,482,40.739355,-73.999318,W 15 St & 7 Ave,6/22/2016 18:53:17,6/22/2016 19:09:21,963,Subscriber
1038668,17075,1969.0,482,40.739355,-73.999318,W 15 St & 7 Ave,1,458,40.751396,-74.005226,11 Ave & W 27 St,6/22/2016 17:19:08,6/22/2016 17:29:01,592,Subscriber
1037607,17075,1957.0,458,40.751396,-74.005226,11 Ave & W 27 St,2,362,40.751726,-73.987535,Broadway & W 37 St,6/22/2016 17:08:05,6/22/2016 17:18:47,642,Subscriber
1029774,17075,1987.0,362,40.751726,-73.987535,Broadway & W 37 St,2,505,40.749013,-73.988484,6 Ave & W 33 St,6/22/2016 14:25:45,6/22/2016 14:28:42,177,Subscriber
1028226,17075,1972.0,505,40.749013,-73.988484,6 Ave & W 33 St,1,442,40.746647,-73.993915,W 27 St & 7 Ave,6/22/2016 13:48:24,6/22/2016 13:52:21,237,Subscriber
1021616,17075,1970.0,442,40.746647,-73.993915,W 27 St & 7 Ave,1,536,40.741444,-73.975361,1 Ave & E 30 St,6/22/2016 10:48:23,6/22/2016 10:58:05,581,Subscriber
1020132,17075,1995.0,536,40.741444,-73.975361,1 Ave & E 30 St,1,379,40.749156,-73.991600,W 31 St & 7 Ave,6/22/2016 10:05:00,6/22/2016 10:14:17,557,Subscriber
1019417,17075,1964.0,379,40.749156,-73.991600,W 31 St & 7 Ave,1,284,40.739017,-74.002638,Greenwich Ave & 8 Ave,6/22/2016 09:49:24,6/22/2016 10:04:03,879,Subscriber


END TEMP

## Inbound bike trips

Same process, just with slightly different inputs...

In [10]:
inbound_station_bikesets = {station_id: [] for station_id in stations.index}

for bike_id in tqdm(np.unique(june_22['bikeid'])):
    first_trip_station = june_22[june_22['bikeid'] == bike_id].sort_values(by='starttime', ascending=False).iloc[0]['start station id']
    inbound_station_bikesets[first_trip_station].append(bike_id)

100%|████████████████████████████████████| 6493/6493 [00:05<00:00, 1089.18it/s]


In [11]:
import functools
inbound_bike_trip_index = dict()

for station_id, bike_id_list in tqdm(inbound_station_bikesets.items()):
    trip_set = []
    for bike_id in bike_id_list:
        trip_set.append(list(june_22[june_22['bikeid'] == bike_id].index.values))
    if len(trip_set) > 0:
        trip_set = functools.reduce(lambda x,y: x+y, trip_set)
    inbound_bike_trip_index[station_id] = trip_set

100%|███████████████████████████████████████| 474/474 [00:02<00:00, 162.49it/s]


And there you have it.

## Trips from station

In [12]:
june_22.head(1)

,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
1004842,25585,1978.0,450,40.762272,-73.987882,W 49 St & 8 Ave,1,334,40.742388,-73.997262,W 20 St & 7 Ave,6/22/2016 00:00:11,6/22/2016 00:13:17,785,Subscriber


In [13]:
outbound_trip_index = dict()

for start_station_id in tqdm(np.unique(june_22['start station id'])):
    trip_ids = list(june_22[june_22['start station id'] == start_station_id].index.values)
    outbound_trip_index[start_station_id] = list(june_22[june_22['start station id'] == start_station_id].index.values)

100%|███████████████████████████████████████| 465/465 [00:00<00:00, 999.94it/s]


## Trips to station

In [14]:
inbound_trip_index = dict()

for end_station_id in tqdm(np.unique(june_22['end station id'])):
    trip_ids = list(june_22[june_22['end station id'] == end_station_id].index.values)
    inbound_trip_index[end_station_id] = list(june_22[june_22['end station id'] == end_station_id].index.values)

100%|███████████████████████████████████████| 470/470 [00:00<00:00, 993.60it/s]


## Loading it into the database

Get that crap into the database!

In [15]:
from pymongo import MongoClient

In [16]:
client = MongoClient("mongodb://localhost:27017")

...right, once again there is weirdness with numpy stuff needing to be cast into Python-native int.

In [17]:
stations.sample(1)

,latitude,longitude,station name,incoming trips,outgoing trips,all trips,kind,bikes outbound,outbound trips,bikes inbound,inbound trips,delta bikes,delta trips
station id,,,,,,,,,,,,,
3182,40.686931,-74.016966,Yankee Ferry Terminal,1,1,2,active,1,1,1,1,0,0


**NOTE**: The following code block generates a new `station-indices` collection document in the database and inserts it. It does *not* replace any pre-existing `station-indices` collection document(s)!

If you run this line multiple times, and do not take care to make sure that you delete any pre-existing documents in the collection first, you will generate multiple copies of the same indices.

To avoid this, if this collection already exists, delete the collection *before* running this cell.

In [20]:
# client['citibike']['station-indices'].insert_one({str(key): inbound_bike_trip_index[key] for key in inbound_bike_trip_index.keys()})
# for key in inbound_bike_trip_index.keys():
for station_id in tqdm(stations.index):
    client['citibike']['station-indices'].insert({
            'station id': str(station_id),
            'tripsets': {
                'inbound bike trip indices': [int(n) for n in inbound_bike_trip_index[station_id]]  if station_id in inbound_bike_trip_index else [],
                'outbound bike trip indices': [int(n) for n in outbound_bike_trip_index[station_id]] if station_id in outbound_bike_trip_index else [],
                'incoming trip indices': [int(n) for n in inbound_trip_index[station_id]] if station_id in inbound_trip_index else [],
                'outgoing trip indices': [int(n) for n in outbound_trip_index[station_id]]  if station_id in outbound_trip_index else []
            }
        })

100%|███████████████████████████████████████| 474/474 [00:00<00:00, 854.01it/s]


And...done!